In [2]:
%serialconnect

Connecting to --port=/dev/ttyUSB0 --baud=115200 
Ready.


In [ ]:
# Small ESP8266

#      RST          |  GPIO1   TX
#       A0          |  GPIO3   RX
#       D0  GPIO16  |  GPIO5   D1  SCL
# SCK   D5  GPIO14  |  GPIO4   D2  SDA
# MISO  D6  GPIO12  |  GPIO0   D3
# MOSI  D7  GPIO13  |  GPIO2   D4  LED
# SS    D8  GPIO15  |          GND
#      3V3          |          5V

# (not able to get the mDNS to work for esp8266)

In [34]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


In [51]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       2
pincontrol   5
pinbuttonled 4
pinbutton    0
boardname    esp8266H

Sent 8 lines (149 bytes) to config.txt.


In [18]:
%serialconnect

serial exception on close write failed: [Errno 5] Input/output error
Connecting to --port=/dev/ttyUSB1 --baud=115200 
Ready.


### Main ESP8266 control switch

In [19]:
%sendtofile main.py

from mqtt_as import config, MQTTClient
import uasyncio as asyncio
import time
from machine import Pin, PWM

fconfig = dict(x.split()  for x in open("config.txt"))

pinled       = Pin(int(fconfig["pinled"]), Pin.OUT)
pincontrol   = Pin(int(fconfig["pincontrol"]), Pin.OUT)
pincontrol.value(0)
pinbuttonled = Pin(int(fconfig["pinbuttonled"]), Pin.OUT)
pinbutton    = Pin(int(fconfig["pinbutton"]), Pin.IN, Pin.PULL_UP)
pwmbuttonled = machine.PWM(pinbuttonled, duty=5)
pinled.value(0)

config['server']    = fconfig["mqttbroker"]
config['ssid'] = fconfig["wifiname"]
config['wifi_pw'] = fconfig["wifipassword"]
config['mqttchannel'] = fconfig["boardname"]
        
boardname         = fconfig["boardname"].encode()
tboardcmd         = boardname+b'/cmd'
tboardcmdon       = boardname+b'/cmd/on/secs'
tonsecs           = boardname+b'/on/secs'
tbuttonsecs       = boardname+b'/button/held/secs'
tboardcmdbuttsecs = boardname+b'/cmd/button/secs'

buttoncontrolonsecs = 10
timeoffticks = 0

outgoingmsgs = { }

async def callbackcmdtask(cmd):
    global buttoncontrolonsecs
    print("executing", [cmd])
    try:
        exec(cmd)
        await client.publish(boardname+'/reply', "1")
    except Exception as e:
        print(e)
        await client.publish(boardname+'/reply/exception', str(e))
    return

def callbackcmd(topic, msg, retained):
    print("callback", topic, msg, retained)
    if topic == tboardcmd:
        aloop.create_task(callbackcmdtask(msg))
    elif topic == tboardcmdon:
        aloop.create_task(callbackcmdtask("controlfunc(cmdsecs=int(%s))"%msg))
    elif topic == tboardcmdbuttsecs:
        aloop.create_task(callbackcmdtask("buttoncontrolonsecs=int(%s)"%msg))
            
async def onconnecttask(client):
    ipnumber = client._sta_if.ifconfig()[0]
    print("ipnumber", ipnumber)
    await client.subscribe(tboardcmdon)
    await client.subscribe(tboardcmd)
    await client.subscribe(tboardcmdbuttsecs)
    print("subscribing to", tboardcmd, tboardcmdon, tboardcmdbuttsecs)
    await client.publish(boardname+b'/ip', ipnumber, retain=True, qos=1)
            
config['subs_cb'] = callbackcmd
config['connect_coro'] = onconnecttask
client = MQTTClient(config)

def controlfunc(cmdsecs=None, sendupdate=False):
    global timeoffticks
    t0 = time.ticks_ms()
    if cmdsecs is not None:
        pincontrol.value(1 if cmdsecs != 0 else 0)
        timeoffticks = t0 + cmdsecs*1000
        sendupdate = True
    if timeoffticks != 0:
        if timeoffticks <= t0:
            pincontrol.value(0)
            timeoffticks = 0
            sendupdate = True
    if sendupdate:
        secondscountdown = ((timeoffticks-t0)/1000 if timeoffticks else 0)
        outgoingmsgs[tonsecs] = secondscountdown
        if pinbutton.value():
            pwmbuttonled.duty(int(min(secondscountdown/buttoncontrolonsecs, 1)*300))

async def controltask():
    while True:
        t = time.ticks_ms()
        await asyncio.sleep_ms(min(timeoffticks-t+100, 1000)  if timeoffticks  else 1000)
        controlfunc(sendupdate=True)

async def buttontask():
    bcurr = True
    while True:
        t = time.ticks_ms()
        bprev, bcurr = bcurr, pinbutton.value()
        if not bcurr: # button on
            if bprev:
                t0 = t
            dt = t - t0
            pwmbuttonled.duty(300 if dt < 2000 else (((dt-2000)//500)%2)*500)
        elif not bprev:
            outgoingmsgs[tbuttonsecs] = dt/1000
            controlfunc(cmdsecs=(buttoncontrolonsecs  if dt < 2000  else 0))

        pinled.value(0 if ((t%1500) < (120 if client.isconnected() else 600)) else 1)
        await asyncio.sleep_ms(50)
        
async def mqtttask():
    while True:
        try:
            await client.connect()
            break
        except OSError as e:
            print("client connect", [e])
    while True:
        while len(outgoingmsgs):
            t, p = outgoingmsgs.popitem()
            print("publish", t, p)
            await client.publish(t, str(p))
        await asyncio.sleep_ms(50)
    
aloop = asyncio.get_event_loop()
aloop.create_task(mqtttask())
aloop.create_task(controltask())
aloop.create_task(buttontask())
aloop.run_forever()



Sent 126 lines (4161 bytes) to main.py.


In [34]:
print(i2c.scan())

[60, 72]


### Remote esp32 button switch

Requires the effective deepsleep() mode not available in the esp8266.

In [36]:
%serialconnect

Connecting to --port=/dev/ttyUSB4 --baud=115200 
Ready.


In [12]:
%sendtofile config.txt

wifiname     BV6000
wifipassword bubblino
mqttbroker   192.168.43.1
pinled       4
pinbutton    14
boardname    bareesp1
deepsleepsec 600

Sent 7 lines (137 bytes) to config.txt.


In [37]:
%sendtofile main.py

import machine, time, esp32

if machine.reset_cause() == machine.DEEPSLEEP_RESET:
    if machine.wake_reason() == machine.EXT0_WAKE:
        state = "button"
    elif machine.wake_reason() == machine.TIMER_WAKE:
        state = "timer"
    else:
        state = "unknown"
else:
    state = "poweron"
print(state)
    
fconfig = dict(x.split()  for x in open("config.txt"))
# p14 yellow, p4 black, G blue    

pinled = machine.Pin(int(fconfig["pinled"]), machine.Pin.OUT)
pinbutton = machine.Pin(int(fconfig["pinbutton"]), machine.Pin.IN)

if state == "poweron" or state == "unknown":
    for i in range(20):
        pinled.value(i%2)
        time.sleep_ms(200)

if state == "button":
    pinled.value(1)
    for timebutton in range(0, 2010, 10):
        if pinbutton.value() == 1:
            break
        time.sleep_ms(10)
    pinled.value(0)
        
if state == "button":
    import network
    si = network.WLAN()
    si.active(True)
    si.connect(fconfig["wifiname"], fconfig["wifipassword"])
    for i in range(20*10):
        if si.isconnected():
            break
        pinled.value((i>20*5) and ((i%(20*2) == 20+18)))
        time.sleep_ms(50)
    else:
        state = "fail"
                     
if state == "button":
    ipnumber = si.ifconfig()[0]
    print(ipnumber)
    
    from umqtt.robust import MQTTClient
    client = MQTTClient(fconfig["boardname"], fconfig["mqttbroker"])
    try:
        client.connect()
        print("mqtt connected")
        client.publish(fconfig["boardname"]+"/button", str(timebutton), qos=1)
        print("published", timebutton)
    except Exception as e:
        print(e)
        state = "fail"

if state == "timer" or state == "button":
    pinled.value(1)
    time.sleep_ms(2)
    pinled.value(0)

if state == "fail":
    for i in range(19):
        pinled.value(i%2)
        time.sleep_ms(50)
    
esp32.wake_on_ext0(pinbutton, esp32.WAKEUP_ALL_LOW)
deepsleepsec = int(fconfig["deepsleepsec"])
print("deepsleep", deepsleepsec)
machine.deepsleep(deepsleepsec*1000)


Sent 74 lines (2023 bytes) to main.py.


In [11]:
%disconnect

attempt to exit paste mode
[\r\x03\x02] b''
Closing serial Serial<id=0x7f89e9149470, open=True>(port='/dev/ttyUSB3', baudrate=115200, bytesize=8, parity='N', stopbits=1, timeout=0.5, xonxoff=False, rtscts=False, dsrdtr=False)
